In [218]:
import numpy as np

def bingo_input(input_file:str):
    """Parse input bingo data - first two lines are called numbers, rest are bingo board arrays"""
    depths = []    
    with open(input_file, 'r') as file:
        # Read first line as the called numbers into list
        callnums = file.readline().strip().split(',')
        callnums = list(map(int, callnums))
        
        # Now create list of arrays for the boards
        bingo_boards = []
        current_board = []
        for line in file:
            if len(line) <= 1:
                if len(current_board) > 1:
                    bingo_boards.append(np.array(current_board).astype(int))
                current_board = []
            else:
                current_board.append(line.strip().split())
        # Catch last line
        bingo_boards.append(np.array(current_board).astype(int))

    return callnums, bingo_boards

In [483]:
class Bingo(object):
        
    def __init__(self, bingoboards:list, callnums:list):
        self.bingoboards = bingoboards
        self.callnums = callnums
        self.current_num_idx = 0 
        self.scoreboards = self.score_board_arrays(len(bingoboards))
        self.win_conds = self.win_condition_arrays()
        self.win_status = self.win_status_list(len(bingoboards))
        self.winning_board = None
        self.winning_marks = None
        self.winning_marked_board = None
        self.winning_num_idx = None
        self.winners = []
        
    def win_condition_arrays(self):
        empty = np.full((5,5),0)
        wins = []
        for i in range(empty.shape[0]):
            win = np.copy(empty)
            win[:,i] = 1
            wins.append(win)
        for i in range(empty.shape[1]):
            win = np.copy(empty)
            win[i,:] = 1
            wins.append(win)
        return wins
    
    def score_board_arrays(self, num:int):
        score_boards = []
        for i in range(num):
            score_boards.append(np.full((5,5),0))
        return score_boards
    
    def win_status_list(self, num:int):
        win_status = []
        for i in range(num):
            win_status.append(False)
        return win_status
    
    def next_num(self):
        if self.current_num_idx < len(self.callnums)-1:
            self.current_num_idx += 1
        
    def callnum_find(self):
        current_num = self.callnums[self.current_num_idx]
        for i, board in enumerate(self.bingoboards):
            found = np.where(board == current_num)
            if found[0].size > 0:
                self.callnum_mark_score(i,found[0][0],found[1][0])
            
    def callnum_mark_score(self, board_idx, mark_0, mark_1):
        if self.win_status[board_idx] == False:
            self.scoreboards[board_idx][mark_0][mark_1] = 1
        else:
            print('skipping, already won')
    
    def callnum_check_win(self):
        for i, score in enumerate(self.scoreboards):
            if self.win_status[i] == False:
                for win in self.win_conds:
                    cond = win*score
                    if (win*score==win).all():
                        self.win_status[i] = True
                        self.winning_num_idx = self.current_num_idx
                        return i                
            
    def find_all(self, mode='first'):
        for i in range(len(self.callnums)):
            print(f'calling number {self.callnums[i]}')
            self.callnum_find()
            board_num = self.callnum_check_win()
            if board_num:
                print('\nWe have a winner!')
                print(f'Winning board: \n{self.bingoboards[board_num]}')
                print(f'Winning marks: \n{self.scoreboards[board_num]}')
                self.winning_board = self.bingoboards[board_num]
                self.marks = self.scoreboards[board_num]
                self.winners.append((callnums[i], board_num, self.bingoboards[board_num],self.scoreboards[board_num]))
                if mode == 'first':
                    break
                elif mode == 'last':
                    pass
                else:
                    raise ValueException('Mode must be "first" or "last"')
            self.next_num()
    
    def find_score(self):
        inverted = 1 - self.marks
        self.winning_marked_board = self.winning_board * inverted
        self.score = np.sum(self.winning_marked_board) * self.callnums[self.winning_num_idx]
        print(f'Marked board: \n{self.winning_marked_board}')
        print(f'Final score (unarmed sum) {np.sum(self.winning_marked_board)} * '\
              f'last called num {self.callnums[self.winning_num_idx]}')
        print(self.score)
        return self.score

In [484]:
# Workflow

# Input
callnums, bingoboards = bingo_input('data/d04_bingo.txt')

# Set up boards
bingo = Bingo(bingoboards, callnums)
# Go through call numbers, break at winning condition
bingo.find_all(mode='first')
# Determine winning score
bingo.find_score()

calling number 46
calling number 79
calling number 77
calling number 45
calling number 57
calling number 34
calling number 44
calling number 13
calling number 32
calling number 88
calling number 86
calling number 82
calling number 91
calling number 97
calling number 89
calling number 1
calling number 48
calling number 31
calling number 18
calling number 10
calling number 55
calling number 74
calling number 24
calling number 11
calling number 80
calling number 78
calling number 28

We have a winner!
Winning board: 
[[19 85 36 73 71]
 [65 62 14 52  3]
 [30 83 44 41  5]
 [55 15  0 61 95]
 [28 13 32 31 88]]
Winning marks: 
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [1 1 1 1 1]]
Marked board: 
[[19 85 36 73 71]
 [65 62 14 52  3]
 [30 83  0 41  5]
 [ 0 15  0 61 95]
 [ 0  0  0  0  0]]
Final score (unarmed sum) 810 * last called num 28
22680


22680

In [485]:
# NOTE: Pt 2 workflow does not yet work - issue is that multiple boards can win in a single round
# yet the function only returns one.  This means that some winning boards will continue to be marked,
# meaning when they finally win they are in the incorrect winning order.

# FIX: change function to return list of winners rather than single board.




# Part2 identical but switch find_all mode to set winning board to last rather than first

callnums, bingoboards = bingo_input('data/d04_bingo.txt')


#Set up boards
bingo = Bingo(bingoboards, callnums)
# Go through call numbers, break at winning condition
bingo.find_all(mode='last')
# Determine winning score
bingo.find_score()

calling number 46
calling number 79
calling number 77
calling number 45
calling number 57
calling number 34
calling number 44
calling number 13
calling number 32
calling number 88
calling number 86
calling number 82
calling number 91
calling number 97
calling number 89
calling number 1
calling number 48
calling number 31
calling number 18
calling number 10
calling number 55
calling number 74
calling number 24
calling number 11
calling number 80
calling number 78
calling number 28

We have a winner!
Winning board: 
[[19 85 36 73 71]
 [65 62 14 52  3]
 [30 83 44 41  5]
 [55 15  0 61 95]
 [28 13 32 31 88]]
Winning marks: 
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [1 1 1 1 1]]
calling number 37

We have a winner!
Winning board: 
[[ 8 13 31 38 59]
 [86 94 55 10  1]
 [81 18 45 48 32]
 [43 25 37 49 67]
 [22 95 11 82 44]]
Winning marks: 
[[0 1 1 0 0]
 [1 0 1 1 1]
 [0 1 1 1 1]
 [0 0 1 0 0]
 [0 0 1 1 1]]
calling number 47

We have a winner!
Winning board: 
[[11 56 41  8 86]
 [53 38 69

0

In [486]:
bingo.winners

[(28,
  60,
  array([[19, 85, 36, 73, 71],
         [65, 62, 14, 52,  3],
         [30, 83, 44, 41,  5],
         [55, 15,  0, 61, 95],
         [28, 13, 32, 31, 88]]),
  array([[0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1]])),
 (37,
  51,
  array([[ 8, 13, 31, 38, 59],
         [86, 94, 55, 10,  1],
         [81, 18, 45, 48, 32],
         [43, 25, 37, 49, 67],
         [22, 95, 11, 82, 44]]),
  array([[0, 1, 1, 0, 0],
         [1, 0, 1, 1, 1],
         [0, 1, 1, 1, 1],
         [0, 0, 1, 0, 0],
         [0, 0, 1, 1, 1]])),
 (47,
  68,
  array([[11, 56, 41,  8, 86],
         [53, 38, 69, 62, 67],
         [32,  6, 35, 24, 66],
         [57, 84, 83, 49,  2],
         [82, 88, 10, 28, 47]]),
  array([[1, 0, 0, 0, 1],
         [0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0],
         [1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1]])),
 (21,
  45,
  array([[60,  6, 79, 29, 12],
         [28, 68, 21, 71, 97],
         [ 3, 24, 34